In [53]:
%run Container.ipynb
%run GameState.ipynb

In [54]:
# Notes:
# if koala's friend was hurt twice, it will activate twice despite n_charges limitation
# in real game, n_charges are shared between shop phase and subsequent battle phase?
#     maybe already done bc of battle copy
# TODO: porcupine

class Battle(GameState):
    def __init__(self, team1, team2, animal_dict=allpacks_animal_dict, current_turn=1, current_tier=1):
        assert isinstance(team1, Team), f'team1, {team1}, must be a Team object'
        assert isinstance(team2, Team), f'team2, {team2}, must be a Team object'
        self.team1 = team1.battle_copy()
        self.team2 = team2.battle_copy()
        self.current_turn = current_turn
        self.current_tier = current_tier
        self.animal_dict = animal_dict
        
    def __str__(self):
        output = ''
        for i in range(max(len(self.team1), len(self.team2))):
            pet1 = self.team1[i] if i < len(self.team1) else ''
            pet2 = self.team2[i] if i < len(self.team2) else ''
            sep = 'v.s.' if i == 0 else ''
            output += f'{pet1!s:50} {sep:10} {pet2!s} \n'
        return output
    
    def battle(self, verbose = False):
        """
        Execute complete battle between two teams. Return winner as 1 or 2, or a draw as 0.
        """
        # startofbattle
        if verbose: print('Battle starts:')
        for pet in self.team1.pets + self.team2.pets:
            pet.trigger_dict['StartOfBattle'] = 1
        self.process_triggers(verbose)

        i = 1
        while len(self.team1) > 0 and len(self.team2) > 0:
            # beforeclash
            if any(['BeforeClash' in abl.trigger for abl in self.team1[0].ability + self.team2[0].ability]):
                if verbose: print(f'Before clash {i}:')
                self.team1[0].trigger_dict['BeforeClash'] = 1
                self.team2[0].trigger_dict['BeforeClash'] = 1
                self.process_triggers(verbose)

            # clash
            if verbose: print(f'Clash {i}:')
            self.clash(verbose)
            self.process_triggers(verbose)
            
            # afterclash
            has_afterclash = False
            for team in [self.team1, self.team2]:
                for pet in team:
                    for abl in pet.abilities[pet.lvl]:
                        has_afterclash += abl.trigger == 'AfterClash'
            if has_afterclash:
                if verbose: print(f'After clash {i}:')
                if len(self.team1) > 0:
                    self.team1[0].trigger_dict['AfterClash'] = 1
                if len(self.team2) > 0:
                    self.team2[0].trigger_dict['AfterClash'] = 1
                self.process_triggers(verbose)
            
            i += 1

        # return winner
        if len(self.team1) == len(self.team2) == 0:
            return 0
        elif len(self.team1) > 0:
            return 1
        elif len(self.team2) > 0:
            return 2

    def clash(self, verbose=False):
        """
        Execute one attack between the front-most animals. Stat changes are in-place.
        """
        if len(self.team1) == 0 or len(self.team2) == 0:
            return
        
        # basic damage
        first1, first2 = self.team1[0], self.team2[0]
        first1.take_damage(first2.deal_damage(), first2.status.name == 'Peanut')
        first2.take_damage(first1.deal_damage(), first1.status.name == 'Peanut')

        # if clashers have chili
        if first1.status.name == 'Chili' and len(self.team2) > 1:
            self.team2[1].take_damage(5)
        if first2.status.name == 'Chili' and len(self.team1) > 1:
            self.team1[1].take_damage(5)

        # update KnockOut trigger
        # TODO: the result of knockout still triggers knockout (implement in process_triggers)
        first1.trigger_dict['KnockOut'] += len([pet for pet in self.team2 if pet.hp <= 0])
        first2.trigger_dict['KnockOut'] += len([pet for pet in self.team1 if pet.hp <= 0])
            
        return